In [170]:
import sys
import os

# os.getcwd() => '/home/genie/Documents/Projekti/cash-flow/jupyter'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Moves up one level to 'cash-flow'
# project_root = '/home/genie/Documents/Projekti/cash-flow'
            
# Add the modules Converters, Model directory to sys.path
sys.path.append(os.path.join(project_root, 'cash_flow', 'util'))


In [171]:
# Imports and engine

import pandas as pd
import numpy as np
import datetime
from sqlalchemy import create_engine

from Converters import date_format

pd.options.mode.copy_on_write = True
engine = create_engine("sqlite:///../data/database.db", echo=False)

In [172]:
FILTER = {"definition_id": 2,
            "date from": pd.to_datetime("2025-05-01"),
            "date through": pd.to_datetime("2025-06-30"),
            "customer": "jones"
         }

class DocType:
    BANK_RECEIPT = 1
    BANK_PAYMENT = 2
    INVOICE_CUSTOMER = 3
    INVOICE_VENDOR = 4
    CREDITNOTE_CUSTOMER = 5
    CREDITNOTE_VENDOR = 6
    PAYROLL = 7

class AccountType:
    BANK_ACCOUNT = 1
    CUSTOMERS_ACCOUNT = 2
    VENDORS_ACCOUNT = 3
    EXPENSES_ACCOUNT = 4
    REVENUES_ACCOUNT = 5
    TAX_ACCOUNT = 6

In [173]:
date_from = FILTER["date from"]
date_through = FILTER["date through"]
next_day_date_through = (pd.to_datetime(date_through) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
previous_day_date_from = (pd.to_datetime(date_from) - pd.Timedelta(days=1)).strftime('%Y-%m-%d')

In [174]:
# Customer Invoices Cleared
doctype = str(DocType.INVOICE_CUSTOMER)
cleared = str(1)
cash_type = "Receipt"

In [175]:
invoices = pd.read_sql_query("SELECT * FROM D01_Documents WHERE type_id = " + doctype + 
                             " AND date_cleared > '" + previous_day_date_from + "' " +
                             " AND date_cleared < '" + next_day_date_through + "'" + 
                             " AND cleared = " + cleared + ""
                             , engine)
doctypes = pd.read_sql_table("D02_DocTypes", engine)
customers = pd.read_sql_query("SELECT id, name FROM B04_Customers", engine)


In [176]:

doctypes.rename(columns={"id": "type_id", "name": "type_name"}, inplace=True)
customers.rename(columns={"id": "customer_id", "name": "customer_name"}, inplace=True)
invoices.rename(columns={"id": "document_id"}, inplace=True)
invoices = pd.merge(invoices, doctypes, on = "type_id", how="left")
invoices = pd.merge(invoices, customers, on = "customer_id", how="left")

In [177]:
invoices["date"]=pd.to_datetime(invoices["date"])
invoices["date_due"]=pd.to_datetime(invoices["date_due"])
invoices["date_planned_clearing"]=pd.to_datetime(invoices["date_planned_clearing"])
invoices["date_cleared"]=pd.to_datetime(invoices["date_cleared"])

In [178]:
invoices.head()

,document_id,source_id,source_key,type_id,date,date_due,date_planned_clearing,date_cleared,priority,number,...,description,amount,currency,amount_LC,memo,cleared,cleared_amount,void,type_name,customer_name
0,22,2,22,3,2025-05-03,2025-05-18,2025-05-18,2025-05-03,100,22,...,Invoice 22,11953.15,EUR,11953.15,None,1,11953.15,0,Rēķins klientam,"Yang, Hutchinson and Drake"
1,46,2,46,3,2025-05-03,2025-05-16,2025-05-16,2025-05-03,100,46,...,Invoice 46,10297.18,EUR,10297.18,None,1,10297.18,0,Rēķins klientam,"Baxter, Vasquez and Gentry"
2,47,2,47,3,2025-05-14,2025-06-12,2025-06-12,2025-05-14,100,47,...,Invoice 47,8796.34,EUR,8796.34,None,1,8796.34,0,Rēķins klientam,Black-Nolan
3,95,2,95,3,2024-12-16,2025-01-13,2025-01-13,2025-05-20,100,95,...,Invoice 95,9628.72,EUR,9628.72,None,1,9628.72,0,Rēķins klientam,"Farrell, Myers and Becker"
4,113,2,113,3,2024-09-10,2024-09-22,2024-09-22,2025-05-12,100,113,...,Invoice 113,4739.23,EUR,4739.23,None,1,4739.23,0,Rēķins klientam,Jones Ltd


In [179]:
filter_customer = FILTER.get("customer")

In [180]:
if filter_customer:
    invoices = invoices[invoices["customer_name"].str.contains(filter_customer, case=False)]

In [181]:
invoices

,document_id,source_id,source_key,type_id,date,date_due,date_planned_clearing,date_cleared,priority,number,...,description,amount,currency,amount_LC,memo,cleared,cleared_amount,void,type_name,customer_name
4,113,2,113,3,2024-09-10,2024-09-22,2024-09-22,2025-05-12,100,113,...,Invoice 113,4739.23,EUR,4739.23,None,1,4739.23,0,Rēķins klientam,Jones Ltd
23,1105,2,1105,3,2023-04-18,2023-05-03,2023-05-03,2025-05-24,100,1105,...,Invoice 1105,8991.68,EUR,8991.68,None,1,8991.68,0,Rēķins klientam,Myers-Jones
28,1287,2,1287,3,2023-07-13,2023-08-12,2023-08-12,2025-05-24,100,1287,...,Invoice 1287,2304.51,EUR,2304.51,None,1,2304.51,0,Rēķins klientam,Myers-Jones
29,1320,2,1320,3,2024-07-05,2024-07-31,2024-07-31,2025-05-12,100,1320,...,Invoice 1320,5495.09,EUR,5495.09,None,1,5495.09,0,Rēķins klientam,Jones Ltd


In [182]:
definition.head()

,definition_id,cash_type,account
0,2,Receipt,6110


In [183]:
filter_definition = FILTER.get("definition_id")

In [184]:
if filter_definition:
    accounts = pd.read_sql_table("B02_Accounts", engine)
    operation_accounts = accounts[accounts["type_id"].isin([AccountType.EXPENSES_ACCOUNT, AccountType.REVENUES_ACCOUNT])]["code"] 
    
    definition = pd.read_sql_table("E01_CashFlowDefinitionAccounts", engine)
    definition.drop(columns="id", inplace=True)
    definition = definition[definition["cash_type"] == cash_type]
    definition = definition[definition["definition_id"] == filter_definition]
    definition = definition[definition["account"].isin(operation_accounts)]
    
    gl = pd.read_sql_table("D03_GeneralLedger", engine)
    gl = gl[gl["document_id"].isin(invoices["document_id"])]
    filtered_invoices = gl[gl["account"].isin(definition["account"])]["document_id"].unique()
    invoices = invoices[invoices["document_id"].isin(filtered_invoices)]
    

In [185]:
invoices

,document_id,source_id,source_key,type_id,date,date_due,date_planned_clearing,date_cleared,priority,number,...,description,amount,currency,amount_LC,memo,cleared,cleared_amount,void,type_name,customer_name
23,1105,2,1105,3,2023-04-18,2023-05-03,2023-05-03,2025-05-24,100,1105,...,Invoice 1105,8991.68,EUR,8991.68,None,1,8991.68,0,Rēķins klientam,Myers-Jones
28,1287,2,1287,3,2023-07-13,2023-08-12,2023-08-12,2025-05-24,100,1287,...,Invoice 1287,2304.51,EUR,2304.51,None,1,2304.51,0,Rēķins klientam,Myers-Jones


In [186]:
invoices.sort_values("date_cleared", inplace=True)

invoices = invoices.reindex(columns=["document_id", "type_name", "number", "customer_name",
                                     "date", "date_due", "date_cleared", 
                                     "amount", "currency", "priority", "memo"])

invoices.columns = ["id", "Tips", "Numurs", "Klients", 
                    "Datums", "Apm. termiņš", "Apm. datums",
                    "Summa", "Valūta", "Prioritāte", "Piezīmes"]

invoices.set_index("id", inplace=True)


In [187]:
invoices

,Tips,Numurs,Klients,Datums,Apm. termiņš,Apm. datums,Summa,Valūta,Prioritāte,Piezīmes
id,,,,,,,,,,
1105,Rēķins klientam,1105,Myers-Jones,2023-04-18,2023-05-03,2025-05-24,8991.68,EUR,100,None
1287,Rēķins klientam,1287,Myers-Jones,2023-07-13,2023-08-12,2025-05-24,2304.51,EUR,100,None
